In [49]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer
)

from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    matthews_corrcoef
)

from datasets import load_dataset, DatasetDict

import evaluate
from evaluate import evaluator

In [42]:
tokenizer = AutoTokenizer.from_pretrained("../training/transformers/gte-base-lazy-teacher")

model_os = AutoModelForSequenceClassification.from_pretrained("../training/transformers/gte-base-lazy-teacher-os").to("mps")
model_us = AutoModelForSequenceClassification.from_pretrained("../training/transformers/gte-base-lazy-teacher-us").to("mps")
model_base = AutoModelForSequenceClassification.from_pretrained("../training/transformers/gte-base-lazy-teacher").to("mps")

In [43]:
data_test = load_dataset('csv', data_files='../../data/test.csv', split='train')

In [44]:
task_evaluator = evaluator("text-classification")

In [45]:
label_mappping = {f"LABEL_{i}": i for i in range(6)}
print(label_mappping)

{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5}


In [48]:
res_base = task_evaluator.compute(
    model_or_pipeline=model_base,
    tokenizer=tokenizer,
    device="mps",
    data=data_test,
    metric=evaluate.combine(["accuracy", "f1", "matthews_correlation", "precision", "recall"]),
    label_mapping=label_mappping,
    input_column="corrected_text",
    label_column="labels",
    random_state=42
)

The model 'NewForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'GPT

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [50]:
f1 = evaluate.load('f1')

EvaluationModule(name: "f1", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel t